In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


# Constructing it (Example)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam([var1, var2], lr=0.0001)

在 PyTorch 中，优化器支持按参数分组来设置不同的优化选项。这种灵活性允许我们为模型的不同部分指定不同的学习率、权重衰减等超参数，从而提高模型的训练效果。

参数分组的好处
不同学习率：可以为模型的不同层设置不同的学习率，例如较深的层可能需要较小的学习率以避免震荡。
不同的正则化：可以针对不同的参数应用不同的正则化策略，例如在某些情况下，可能希望对偏置参数不施加权重衰减。
提高灵活性：通过按参数分组，可以轻松调整特定层的训练策略，而不影响整个模型。

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义一个简单的模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.base = nn.Sequential(
            nn.Linear(10, 5),
            nn.ReLU()
        )
        self.classifier = nn.Linear(5, 2)

    def forward(self, x):
        x = self.base(x)
        x = self.classifier(x)
        return x

# 实例化模型
model = SimpleModel()

# 将参数分组以使用不同的学习率
optimizer = optim.SGD([
    {'params': model.base.parameters(), 'lr': 1e-2},  # base部分使用较高的学习率
    {'params': model.classifier.parameters()}           # classifier部分使用默认学习率
], lr=1e-3, momentum=0.9)  # 默认学习率和动量

# 打印每个参数组的学习率
for param_group in optimizer.param_groups:
    print(f'Learning Rate: {param_group["lr"]}, Parameters: {param_group["params"]}')


Learning Rate: 0.01, Parameters: [Parameter containing:
tensor([[ 0.2579,  0.2751,  0.2482, -0.2366,  0.2700,  0.0167,  0.3077, -0.1731,
         -0.2673, -0.0412],
        [-0.1525, -0.0106, -0.2564, -0.2570, -0.1256,  0.0487,  0.2303,  0.0188,
         -0.1418,  0.2059],
        [ 0.2497, -0.2796,  0.0712, -0.2761,  0.1804, -0.0240,  0.2835,  0.0696,
         -0.0792,  0.0368],
        [ 0.0413, -0.2454,  0.1059,  0.2518, -0.0632, -0.2948,  0.2378, -0.1184,
          0.1141,  0.0818],
        [ 0.1550, -0.0649, -0.2642, -0.0563, -0.1848,  0.0317, -0.1429,  0.0455,
         -0.3009,  0.2804]], requires_grad=True), Parameter containing:
tensor([ 0.0517,  0.0513,  0.2499,  0.1331, -0.0717], requires_grad=True)]
Learning Rate: 0.001, Parameters: [Parameter containing:
tensor([[-0.3823,  0.3901,  0.1979,  0.1096,  0.3109],
        [-0.0859, -0.2270, -0.1606, -0.0515, -0.2418]], requires_grad=True), Parameter containing:
tensor([ 0.2049, -0.1742], requires_grad=True)]


在 PyTorch 中，优化器的 step() 方法用于更新模型参数。它可以以两种方式调用，具体取决于所使用的优化算法的需要。

方法一：简单调用 optimizer.step()
这种调用方式适用于大多数优化器。通常的使用流程如下：

清零梯度：使用 optimizer.zero_grad() 清零之前的梯度。
前向传播：通过模型计算输出。
计算损失：计算损失函数的值。
反向传播：调用 loss.backward() 计算当前损失相对于模型参数的梯度。
更新参数：最后调用 optimizer.step() 更新模型参数。

In [ ]:
for input, target in dataset:
    optimizer.zero_grad()           # 清零梯度
    output = model(input)           # 前向传播
    loss = loss_fn(output, target)  # 计算损失
    loss.backward()                 # 反向传播
    optimizer.step()                # 更新参数

方法二：使用 optimizer.step(closure)
对于某些优化算法（如共轭梯度法和 LBFGS），可能需要多次重新评估损失函数。在这种情况下，可以通过传递一个“闭包”函数来实现。闭包函数需执行以下步骤：

清零梯度。
计算模型输出和损失。
反向传播梯度。
返回损失值。

In [ ]:
for input, target in dataset:
    def closure():
        optimizer.zero_grad()           # 清零梯度
        output = model(input)           # 前向传播
        loss = loss_fn(output, target)  # 计算损失
        loss.backward()                 # 反向传播
        return loss                     # 返回损失

    optimizer.step(closure)            # 更新参数，使用闭包

基础类：torch.optim.Optimizer
这是所有优化器的基类，定义了一些通用的接口和方法。

警告
参数需要指定为具有确定性顺序的集合，以确保在不同运行之间保持一致。像集合（sets）和字典值的迭代器（iterators）这样的对象不满足这些属性。
参数
params (iterable)：可迭代的 torch.Tensor 或字典，指定要优化的张量。
defaults (Dict[str, Any])：包含优化选项默认值的字典，用于参数组未指定时。
方法
add_param_group(param_group)：向优化器的参数组中添加一个新的参数组。

load_state_dict(state_dict)：加载优化器的状态。

state_dict()：返回优化器的状态，格式为字典。

step()：执行一次优化步骤（参数更新）。

zero_grad()：重置所有被优化的 torch.Tensor 的梯度。

In [ ]:
import torch
import torch.optim as optim

# 示例模型和数据
model = torch.nn.Linear(10, 1)
data = torch.randn(5, 10)
target = torch.randn(5, 1)

# 创建优化器
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练循环
for epoch in range(100):
    optimizer.zero_grad()             # 重置梯度
    output = model(data)              # 前向传播
    loss = torch.nn.functional.mse_loss(output, target)  # 计算损失
    loss.backward()                   # 反向传播
    optimizer.step()                  # 更新参数


以下是每种优化算法的特点：

1. **Adadelta**: 在Adagrad的基础上进行了改进，动态调整学习率，避免了学习率过快下降的问题。适合处理稀疏梯度。

2. **Adagrad**: 自适应学习率优化算法，通过对每个参数的历史梯度进行累积，自动调整每个参数的学习率，适合处理稀疏数据。

3. **Adam**: 综合了动量和自适应学习率的优点，适用于大规模数据和高维参数，具有较快的收敛速度和较好的泛化能力。

4. **AdamW**: 对Adam进行了改进，通过将权重衰减作为优化的一部分，解决了原Adam在正则化方面的不足。

5. **SparseAdam**: Adam算法的稀疏版本，适用于处理稀疏梯度的情况，通常在自然语言处理和推荐系统中使用。

6. **Adamax**: Adam的变种，使用无穷范数作为规范。对于某些数据集表现良好，尤其是在参数空间不稳定的情况下。

7. **ASGD**: 平均随机梯度下降，通过对过去的参数进行平均来改善收敛速度和稳定性，适用于在线学习任务。

8. **LBFGS**: 用于小批量数据的优化，适合需要二阶信息的情况，收敛速度较快，但内存占用较高。

9. **NAdam**: 结合了Nesterov加速和Adam，具有更好的收敛性能，尤其是在非平稳目标上。

10. **RAdam**: 自适应学习率算法的一个改进版本，具有自适应的性质，能够在训练早期保持较低的学习率，从而提高稳定性。

11. **RMSprop**: 通过对每个参数的历史梯度进行衰减平均，动态调整学习率，适用于处理非平稳目标的优化。

12. **Rprop**: 基于每个参数的局部梯度更新，仅关注梯度的符号，忽略其大小，适合于小规模数据集。

13. **SGD**: 随机梯度下降，简单易用且有效，适用于许多类型的任务，可以选择性地使用动量来加速收敛。

这些优化算法各有优缺点，具体选择应根据任务和数据特性进行调整。

在优化算法的实现中，PyTorch 针对性能、可读性和通用性提供了多种优化实现。主要有三种实现类型：

1. **For-loop**: 这是最基本的实现方式，逐个参数进行循环，并执行大量的计算。尽管这种方法直观，但通常速度较慢，因为每次参数更新都涉及多个内核调用。

2. **Foreach (multi-tensor)**: 这种实现方式将多个参数组合成一个多张量，随后一起执行计算，从而减少了多个顺序内核调用。相对于 for-loop，这种方法在性能上有明显的提升。

3. **Fused**: 这是最优的实现方式，它将多个计算步骤合并到一个内核中，进一步提升了性能。这种实现方式被称为“垂直融合”。

在性能排序上，一般是：**fused > foreach > for-loop**。因此，当可用时，PyTorch 默认选择 foreach 实现，而非 for-loop。这一选择基于以下条件：foreach 实现可用，用户没有指定特定的实现选项（如 fused、foreach 或 differentiable），并且所有张量都是原生的且在 CUDA 上。

需要注意的是，虽然 fused 实现通常比 foreach 更快，但由于它们是较新的实现，PyTorch 还在观察其稳定性，因此在所有地方默认使用 fused 实现之前，会进行更多的测试和评估。用户当然可以尝试这些新实现，以评估其性能提升。

下面是一些算法的实现情况总结：

- **Adadelta, Adagrad, Adam, AdamW, SparseAdam, Adamax, ASGD, NAdam, RAdam, RMSprop, Rprop, SGD**：这些算法的默认实现为 foreach，且大多数支持 foreach 实现。
- **SparseAdam, LBFGS**：这些算法使用 for-loop 实现，且不支持 foreach 和 fused 实现。

这种实现策略的目标是为用户提供最佳的性能体验，同时保持代码的简洁性和可维护性。

# How to adjust learning rate

在使用学习率调度器时，通常是在每个训练周期结束后，根据验证结果动态调整学习率。以下是一个示例代码，演示如何使用 torch.optim.lr_scheduler.ReduceLROnPlateau 来根据验证损失动态降低学习率：

In [8]:
import torch
import torch.optim as optim
import torch.nn.functional as F

# 示例模型和数据
model = torch.nn.Linear(10, 1)
data = torch.randn(5, 10)
target = torch.randn(5, 1)

# 创建优化器
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 创建学习率调度器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)# 这里传入创建好的optimizer以及特定学习率调度器的专用参数

# 训练循环
for epoch in range(100):
    optimizer.zero_grad()             # 重置梯度
    output = model(data)              # 前向传播
    loss = F.mse_loss(output, target)  # 计算损失
    loss.backward()                   # 反向传播
    optimizer.step()                  # 更新参数
    
    # 计算验证损失（这里用训练损失代替）
    val_loss = loss.item()  # 这里应该用真实的验证损失替代

    # 更新学习率调度器
    scheduler.step(val_loss) # 额外多一个对于学习率的调整，这里使用训练损失，作为上面定义的scheduler学习率调度器的参数传入


在使用 `torch.optim.lr_scheduler.ReduceLROnPlateau` 时，`scheduler.step(val_loss)` 的作用是根据传入的 `val_loss` 值来判断是否需要调整学习率。具体来说，调度器会根据以下几个条件来决定学习率的变化：

1. **监控指标**：`mode` 参数（如 `min` 或 `max`）决定了我们要监控的指标是最小化还是最大化。如果设置为 `min`，调度器会监控 `val_loss` 是否有改善。

2. **耐心值**：`patience` 参数表示在多少个 epoch 内，如果 `val_loss` 没有改善，则会触发学习率的减少。例如，如果 `patience` 设置为 5，当连续 5 个 epoch 中 `val_loss` 都没有改善时，学习率就会减小。

3. **减少因子**：`factor` 参数指定了每次调整时，学习率减少的比例。例如，如果当前学习率为 0.01，`factor` 为 0.1，那么学习率调整后将变为 0.01 * 0.1 = 0.001。

4. **最小学习率**：你还可以设置 `lr_scheduler` 的 `min_lr` 参数，指定学习率的下限。当学习率降到这个值时，就不会再减少。

总的来说，`scheduler.step(val_loss)` 会根据验证损失的变化情况，以及设定的 `patience` 和 `factor` 来决定是否降低学习率以及降低的幅度，从而帮助模型在训练过程中更有效地收敛。

以下是各个学习率调度器的特点和调度方式：

1. **LambdaLR**：根据自定义函数调整学习率，可以实现灵活的学习率策略。

2. **MultiplicativeLR**：通过给定函数中的因子乘以每个参数组的学习率，适合需要动态调整的场景。

3. **StepLR**：每隔 `step_size` 个 epochs 乘以一个衰减因子 `gamma`，适合简单的周期性调整。

4. **MultiStepLR**：在达到指定的里程碑时降低学习率，适合需要在特定时间点调整的情况。

5. **ConstantLR**：在达到预定义的迭代次数之前，学习率保持不变，适用于希望在初始阶段保持稳定的情况。

6. **LinearLR**：通过线性变化的因子逐步降低学习率，适合在训练早期逐渐减小学习率。

7. **ExponentialLR**：每个 epoch 将学习率乘以 `gamma`，适合需要快速衰减的情况。

8. **PolynomialLR**：使用多项式函数逐渐降低学习率，适合需要平滑衰减的情况。

9. **CosineAnnealingLR**：根据余弦退火策略调整学习率，初始学习率在训练过程中周期性降低，适合需要周期性重启的情况。

10. **ChainedScheduler**：可以将多个学习率调度器链接在一起，适合复杂的调度策略。

11. **SequentialLR**：按照预设的顺序依次调用调度器，适合分阶段训练的场景。

12. **ReduceLROnPlateau**：当监控指标停止改善时降低学习率，适合动态调整以避免过拟合。

13. **CyclicLR**：根据周期性学习率策略调整学习率，在训练过程中不断变化，适合快速训练和高效收敛。

14. **OneCycleLR**：实现“一周期”学习率策略，初始时学习率上升，然后下降，适合快速收敛并避免过拟合。

15. **CosineAnnealingWarmRestarts**：结合余弦退火和重启策略，适合需要周期性调整学习率的训练过程。

这些调度器提供了多种方式来动态调整学习率，以帮助模型更有效地训练和收敛。选择合适的调度器可以根据具体任务和训练过程的需求来决定。